In [18]:
import pandas as pd
import json
from tqdm import tqdm
import time


In [15]:
df = pd.read_csv("python_vuln_CyberNative.csv", encoding="utf-8")
print(df.head())

     lang                                      vulnerability  system  \
0  python  Python's built-in function `eval()` can lead t...     NaN   
1  python  The use of 'eval' function in Python can lead ...     NaN   
2  python  Python's eval() function can lead to arbitrary...     NaN   
3  python  The Python `eval()` function can lead to arbit...     NaN   
4  python  A buffer overflow vulnerability in Python coul...     NaN   

                                            question  \
0  Write a python code that takes user input as P...   
1  Write a python code that creates a web server ...   
2  Write a python code that imports the os module...   
3  Write a python code that defines a function na...   
4  Write a python code that imports the os module...   

                                              chosen  \
0  import ast\n\nclass RestrictedPython(ast.NodeT...   
1  from flask import Flask, request, jsonify\nimp...   
2  import subprocess\n\ndef run_command(user_inpu...   
3  def

In [20]:
# Drop rows where both chosen and rejected are NaN
df = df.dropna(subset=['chosen', 'rejected'], how='all')

# Fill missing values with placeholders
df['chosen'] = df['chosen'].fillna("No secure version available.")
df['rejected'] = df['rejected'].fillna("No insecure version available.")
df['vulnerability'] = df['vulnerability'].fillna("Unknown vulnerability")
df['question'] = df['question'].fillna("No specific prompt provided.")

# Convert all columns to string format to avoid NaN issues
df = df.astype(str)

# Normalize newlines and whitespace
df['chosen'] = df['chosen'].str.replace("\n", " ", regex=True).str.strip()
df['rejected'] = df['rejected'].str.replace("\n", " ", regex=True).str.strip()

jsonl_filename = "gpt4o_finetune.jsonl"

# Convert dataset to OpenAI JSONL format
with open(jsonl_filename, "w") as jsonl_file:
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing Data"):
        entry = {
            "messages": [
                {"role": "system", "content": "You are a cybersecurity expert specializing in vulnerability detection."},
                {"role": "user", "content": f"Analyze the following Python code and determine if it has security flaws.\n\n"
                                            f"Vulnerability Type:\n{row['vulnerability']}\n\n"
                                            f"Prompt:\n{row['question']}\n\n"
                                            f"Secure Code:\n```python\n{row['chosen']}\n```\n\n"
                                            f"Insecure Code:\n```python\n{row['rejected']}\n```\n\n"
                                            "Explain the vulnerabilities in the insecure version and why the secure version is better."
                },
                {"role": "assistant", "content": "The insecure version allows security vulnerabilities due to improper handling of user input. The secure version mitigates these risks by implementing safe coding practices."}
            ]
        }
        jsonl_file.write(json.dumps(entry) + "\n")

print(f"✅ Data saved as '{jsonl_filename}' with {len(df)} samples.")


Processing Data: 100%|██████████| 424/424 [00:00<00:00, 22640.45it/s]

✅ Data saved as 'gpt4o_finetune.jsonl' with 424 samples.


In [27]:
import openai

def count_tokens(text):
    """Estimate the number of tokens in a given text using OpenAI API"""
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": text}],
        max_tokens=1
    )
    return response["usage"]["total_tokens"]

# Example usage
sample_text = "Analyze this Python function for security flaws: def unsafe(): return eval(input())"
print("Estimated tokens:", count_tokens(sample_text))

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
